In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import re
import string
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
amazon_best_50_books = pd.read_csv("/kaggle/input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv")

In [ ]:
amazon_best_50_books.head()

### Following are the 7 features in our dataset :-
#### Name (string),Author (string),User Rating (float),Reviews (int),Price (int),Year (int), Genre (string)

In [ ]:
print("There are total {} books in our dataset".format(amazon_best_50_books.shape[0]))

### Number of unique Authors

In [ ]:
print("There are {} unique authors in our dataset".format(len(amazon_best_50_books["Author"].unique())))

### There are total 248 unique authors in the dataset but 550 books in our dataset. That means that some of the authors have written multiple books.
### Let us look at the authors who have written multiple books.

In [ ]:
num_authos_more_than_1_book = len(amazon_best_50_books["Author"].value_counts().keys()[amazon_best_50_books["Author"].value_counts()>1])
print("Number of authors with more than 1 book in list is {}".format(num_authos_more_than_1_book))

### Let us see what all Genres of books do we have in our dataset and see a distribution of that

In [ ]:
amazon_best_50_books["Genre"].value_counts()

In [ ]:
sns.countplot(amazon_best_50_books["Genre"])

### As the numbers and the graph suggest that number of Non Fiction books are greater than Fiction books in our top 50 best rated books in Amazon from 2009-2019. 

### But which set of books are rated better?? We can have a sense of that by looking at a distribution of the rating for these two different Genre of books

In [ ]:
sns.boxplot(x=amazon_best_50_books["Genre"],y=amazon_best_50_books["User Rating"])

### The above visualization shows that though more number of Non Fiction books feature in the top 50 best rated books in Amazon from 2009-2019, yet the Fiction books are rated a bit highly by the readers.

### Now let us see what is the average price of all the books which are in our dataset.

In [ ]:
print("Mean value of the price of the books : {}".format(amazon_best_50_books["Price"].mean()))
print("Median value of the price of the books : {}".format(amazon_best_50_books["Price"].median()))

In [ ]:
sns.distplot(amazon_best_50_books["Price"])

### Most of the books are low priced around 13 which is visually confirmed by the right skew in the histogram.

### Are low priced books (lesser than or equal to the 75th percentile value for price) better rated and have more reviews??

In [ ]:
amazon_best_50_books.loc[amazon_best_50_books["Price"]<=np.percentile(amazon_best_50_books["Price"],75),"Category"] = "low priced"
amazon_best_50_books.loc[amazon_best_50_books["Price"]>np.percentile(amazon_best_50_books["Price"],75),"Category"] = "high priced"

In [ ]:
sns.boxplot(x=amazon_best_50_books['Category'],y=amazon_best_50_books['Reviews'],hue=amazon_best_50_books["Genre"])

### Thus we see on an average the Ficton books are having more Reviews than the Non Fiction books with the difference between giving reviews for high priced books of the two categories being more than for the low priced books of the two categories

### High priced Non Fiction book have least number of Reviews (having no outlier)
### Low priced Fiction books have a wide range of number of Reviews with many outliers.

### We can summarize that Fiction lovers buying low priced books are better Reviewers.
### Non Fiction lovers buying high priced books are the worst Reviewers.

In [ ]:
sns.boxplot(x=amazon_best_50_books['Category'],y=amazon_best_50_books['User Rating'],hue=amazon_best_50_books["Genre"])

### Fiction lovers rate their books highly irrespective of the price of the book.

### Let us see if we can find any trend between Year of publishing of book and its Reviews and Rating

In [ ]:
sns.set(rc={'figure.figsize':(15,8.27)})
sns.boxplot(x=amazon_best_50_books['Genre'],y=amazon_best_50_books['Reviews'],hue=amazon_best_50_books["Year"])

### We can see an increasing trend among Non Fiction readers giving Reviews from 2009-2019. It can suggest that Non Fiction books have been more popular over time.

### For Fiction readers the trend is somewhat not linear and follow a sine curve approximately. It suggests that at around 2013,2014 and 2015 the popularity of the Fiction Novel peaked if we take Reviews being a metric for the popularity of a book.



In [ ]:
#sns.set(rc={'figure.figsize':(15,8.27)})
sns.boxplot(x=amazon_best_50_books['Genre'],y=amazon_best_50_books['User Rating'],hue=amazon_best_50_books["Year"])

### Let us find the best book based on Reviews and rating for the different years for the two genres

### List of Best Books and its Authors based on User Rating and Reviews for the different categories

In [ ]:
from tqdm.auto import tqdm
d = amazon_best_50_books.copy()
for year in tqdm(range(2009,2020)):
    for i in ["Fiction","Non Fiction"]:
        best_rated = np.max(d.loc[((d['Year']==year) & (d['Genre']==i)),"User Rating"])
        best_review = np.max(d.loc[((d['Year']==year) & (d['Genre']==i) & (d["User Rating"]==best_rated)),"Reviews"])
        best_book = d.loc[((d['Year']==year) & (d['Genre']==i) & (d["User Rating"]==best_rated) & (d["Reviews"]==best_review)),"Name"].unique()
        author = d.loc[((d['Year']==year) & (d['Genre']==i) & (d["User Rating"]==best_rated) & (d["Reviews"]==best_review)),"Author"].unique()
        print("--------------------------------------")
        print("Year : {}".format(year))
        print("Genre : {}".format(i))
        print("Book Name : {}".format(best_book[0]))
        print("Author Name : {}".format(author[0]))
        print("--------------------------------------")